# Analysis for a 3 quantum damping channel discrimination

In [1]:
import os,sys,inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0,parentdir) 

from typing import cast, Tuple, List
from qcd import load_object_from_file
from qcd.typings.configurations import OptimalConfigurations
from qcd.configurations import OneShotConfiguration
import math

In [2]:
results = load_object_from_file(name='20210414b_C2_A2_1000_10000_3_1_20', path='../demo/data/')

In [ ]:
class OptimalConfigurations(TypedDict, total=False):
    eta_groups: List[List[float]]
    best_algorithm: List[str]
    probabilities: List[float]
    configurations: List[ChannelConfiguration]
    number_calls_made: List[int]
    legacy: bool

return {'state_probability': self._state_probability if hasattr(self, 'state_probability') else self._theta,
'angle_rx': self._angle_rx,
'angle_ry': self._angle_ry,
'eta_group': self._eta_group,
}

In [3]:
def get_number_eta_pairs(eta_groups)->int:
    eta_third = eta_groups[0][2]
    number_eta_pairs = 0
    for eta_group in eta_groups:
        if eta_group[2] != eta_third:
            return number_eta_pairs
        number_eta_pairs += 1

In [104]:
def get_etas_and_probabilities_from_optimal_results(optimal_results: OptimalConfigurations)-> List[dict]:
    """ From the optimal results, obtain the etas and the result probabilities for each 3-eta execution.
        Return a list of etas used (in degrees) and the probabilities with the associated eta pair.
    """
    probabilities = []
    eta_pairs = []
    etas_third_channel = []
    
    for configuration in optimal_results['configurations']:
        eta_pairs.append((int(math.degrees(cast(OneShotConfiguration, configuration).eta_group[0])),
                         int(math.degrees(cast(OneShotConfiguration, configuration).eta_group[1]))))
        etas_third_channel.append(int(math.degrees(cast(OneShotConfiguration, configuration).eta_group[2])))
        probabilities.append(cast(OneShotConfiguration, configuration).state_probability)
    
    # print(etas_third_channel)
    # print(eta_pairs)
    number_eta_pairs = get_number_eta_pairs(eta_groups=optimal_results['eta_groups'])
    number_third_channels = int(len(optimal_results['eta_groups']) / number_eta_pairs)
    
    one_eta_pair_result = {
        'probabilities': [],
        'etas_third_channel': [],
        'eta_pair': (0,0)
    }
    two_eta_pair_result = {
        'probabilities': [],
        'etas_third_channel': [],
        'eta_pair': (0,0)
    }
    three_eta_pair_result = {
        'probabilities': [],
        'etas_third_channel': [],
        'eta_pair': (0,0)
    }
    four_eta_pair_result = {
        'probabilities': [],
        'etas_third_channel': [],
        'eta_pair': (0,0)
    }
    five_eta_pair_result = {
        'probabilities': [],
        'etas_third_channel': [],
        'eta_pair': (0,0)
    }
    six_eta_pair_result = {
        'probabilities': [],
        'etas_third_channel': [],
        'eta_pair': (0,0)
    }
    results_to_plot = [[[],[],(0,0)]] * number_eta_pairs
    results_one = [[],[],(0,0)]
    
    for idx, (probability, eta_third, eta_pair) in enumerate(zip(probabilities, etas_third_channel, eta_pairs)):
        # results_to_plot[idx % number_eta_pairs]['probabilities'].append(probability)
        # results_to_plot[idx % number_eta_pairs]['etas_third_channel'].append(eta_third)
        # results_to_plot[idx % number_eta_pairs]['eta_pair'] = eta_pair
        if idx % 6 == 0: 
            one_eta_pair_result['probabilities'].append(probability)
            one_eta_pair_result['etas_third_channel'].append(eta_third)
            one_eta_pair_result['eta_pair'] = eta_pair
            results_to_plot[0][0] = 'heelo'
            results_to_plot[0][1] = 'bye'
            results_to_plot[0][2] = eta_pair
        if idx % 6 == 1: 
            two_eta_pair_result['probabilities'].append(probability)
            two_eta_pair_result['etas_third_channel'].append(eta_third)
            two_eta_pair_result['eta_pair'] = eta_pair
        if idx % 6 == 2: 
            three_eta_pair_result['probabilities'].append(probability)
            three_eta_pair_result['etas_third_channel'].append(eta_third)
            three_eta_pair_result['eta_pair'] = eta_pair
        if idx % 6 == 3: 
            four_eta_pair_result['probabilities'].append(probability)
            four_eta_pair_result['etas_third_channel'].append(eta_third)
            four_eta_pair_result['eta_pair'] = eta_pair
        if idx % 6 == 4: 
            five_eta_pair_result['probabilities'].append(probability)
            five_eta_pair_result['etas_third_channel'].append(eta_third)
            five_eta_pair_result['eta_pair'] = eta_pair
        if idx % 6 == 5: 
            six_eta_pair_result['probabilities'].append(probability)
            six_eta_pair_result['etas_third_channel'].append(eta_third)
            six_eta_pair_result['eta_pair'] = eta_pair
    
    # return [one_eta_pair_result, two_eta_pair_result, three_eta_pair_result, four_eta_pair_result, five_eta_pair_result, six_eta_pair_result]
    # results_to_plot.append(results_one)
    return results_to_plot
    

In [105]:
parsed_results = get_etas_and_probabilities_from_optimal_results(optimal_results=results)

In [106]:
len(parsed_results)

6

In [107]:
parsed_results

[['heelo', 'bye', (0, 29)],
 ['heelo', 'bye', (0, 29)],
 ['heelo', 'bye', (0, 29)],
 ['heelo', 'bye', (0, 29)],
 ['heelo', 'bye', (0, 29)],
 ['heelo', 'bye', (0, 29)]]

In [ ]:
[parsed_result['eta_pair'] for parsed_result in parsed_results]

[(0, 29), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0)]

In [85]:
len(parsed_results[0]['etas_third_channel'])

40

In [87]:
parsed_results[0]['etas_third_channel']

[0,
 0,
 4,
 4,
 9,
 9,
 14,
 14,
 18,
 18,
 23,
 23,
 28,
 28,
 33,
 33,
 37,
 37,
 42,
 42,
 47,
 47,
 52,
 52,
 56,
 56,
 61,
 61,
 66,
 66,
 71,
 71,
 75,
 75,
 80,
 80,
 85,
 85,
 90,
 90]

In [89]:
parsed_results[0]['probabilities']

[0.8333333333333333,
 0.8333333333333333,
 0.8337905807041609,
 0.8337905807041609,
 0.8333333333333333,
 0.8333333333333333,
 0.8333333333333333,
 0.8333333333333333,
 0.8333333333333333,
 0.8333333333333333,
 0.7222222222222221,
 0.7222222222222221,
 0.8333332636416139,
 0.8333332636416139,
 0.8333333333333333,
 0.8333333333333333,
 0.5,
 0.5,
 0.796245491032871,
 0.796245491032871,
 0.8331809175430573,
 0.8331809175430573,
 0.8209876543209875,
 0.8209876543209875,
 0.8333333333333333,
 0.8333333333333333,
 0.7222222222222221,
 0.7222222222222221,
 0.7962962937151215,
 0.7962962937151215,
 0.7962962962962963,
 0.7962962962962963,
 0.8333333333333333,
 0.8333333333333333,
 0.7962962962962963,
 0.7962962962962963,
 0.7222222222222221,
 0.7222222222222221,
 0.7222222222222221,
 0.7222222222222221]

In [44]:
one_eta_pair_result = {
        'probabilities': [],
        'etas_third_channel': [],
        'eta_pair': (0,0)
    }

In [45]:
[one_eta_pair_result]*6

[{'probabilities': [], 'etas_third_channel': [], 'eta_pair': (0, 0)},
 {'probabilities': [], 'etas_third_channel': [], 'eta_pair': (0, 0)},
 {'probabilities': [], 'etas_third_channel': [], 'eta_pair': (0, 0)},
 {'probabilities': [], 'etas_third_channel': [], 'eta_pair': (0, 0)},
 {'probabilities': [], 'etas_third_channel': [], 'eta_pair': (0, 0)},
 {'probabilities': [], 'etas_third_channel': [], 'eta_pair': (0, 0)}]